## Requirements

In [ ]:
!mkdir /root/.kaggle/
!pip install pandas
import pandas as pd
import numpy as np
import os
import json


# Installing the Kaggle package
!pip install kaggle 

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"something","key":"keykeykey"}

# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


## Q1

In [ ]:
# download the dataset from Kaggle and unzip it
!kaggle datasets download -dcsanhueza/the-marvel-universe-social-network -p ./datasets/marvel/
!unzip ./datasets/marvel/*.zip  -d ./datasets/marvel/

edges_df = pd.read_csv('/content/datasets/marvel/edges.csv')
net_df = pd.read_csv('/content/datasets/marvel/hero-network.csv')
nodes_df = pd.read_csv('/content/datasets/marvel/nodes.csv')

the-marvel-universe-social-network.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ./datasets/marvel/the-marvel-universe-social-network.zip
replace ./datasets/marvel/edges.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./datasets/marvel/edges.csv  
  inflating: ./datasets/marvel/hero-network.csv  
  inflating: ./datasets/marvel/nodes.csv  


### 1

In [ ]:
net_df['hero1'], net_df['hero2'] = (net_df['hero1'].str.strip(), net_df['hero2'].str.strip())
nodes_df['node'] = nodes_df['node'].str.strip()

hero_nodes = nodes_df[nodes_df['type'] == 'hero']
hero_nodes = list(set(hero_nodes['node'].values))

# no self loops
net_df = net_df[net_df['hero1'] != net_df['hero2']]
net_df = net_df.drop_duplicates(['hero1','hero2'])

friendly = set(net_df['hero1']) | set(net_df['hero2'])
print('\n\t1. The number of heroes without friends is:', len(hero_nodes) - len(friendly))


	1. The number of heroes without friends is: 18


## 2

In [ ]:
!pip install python-Levenshtein


     |████████████████████████████████| 51kB 1.9MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144791 sha256=602c96c6bba4612f6830a6434a9a6ce714419b47f3e9479b04710ee4c43419ae
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [ ]:
import Levenshtein as lv


real_spidy = 'SPIDER-MAN/PETER PAR'
hero2 = 'CAPTAIN AMERICA'


almost_spidy = [i for i in all_heros.union(hero_nodes) if lv.distance(real_spidy.lower().split('/')[0], i.lower().split('/')[0]) < 2]

captain_friends = net_df[(net_df['hero1'] == hero2) | (net_df['hero2'] == hero2)]
spider_friends = net_df[(net_df['hero1'].isin(almost_spidy)) | (net_df['hero2'].isin(almost_spidy))]

captain_friends = pd.DataFrame(pd.unique(captain_friends[['hero1', 'hero2']].values.ravel('K'))).rename(columns={0:'friends'})
spider_friends = pd.DataFrame(pd.unique(spider_friends[['hero1', 'hero2']].values.ravel('K'))).rename(columns={0:'friends'})

captain_friends = pd.DataFrame(captain_friends[(captain_friends['friends'] != hero2) & (~captain_friends['friends'].isin(almost_spidy))])
spider_friends = pd.DataFrame(spider_friends[(spider_friends['friends'] != hero2) & (~spider_friends['friends'].isin(almost_spidy))])

c_friends = []
for i in captain_friends['friends']:
  for j in spider_friends['friends']:
    if i == j:
      c_friends.append(i)
print(len(c_friends))

921


## 3

In [ ]:
name = [hero_nodes[0], list(hero_nodes)[0]]
for i in hero_nodes:
  if len(i) < len(name[0]):
    name[0] = i
  
  if len(i) > len(name[1]):
    name[1] = i

shortests = list(filter(lambda x: len(x) == len(name[0]), hero_nodes))
longests = list(filter(lambda x: len(x) == len(name[1]), hero_nodes))

print('Shortest names:', shortests, '\nLongest names:', longests)

Shortest names: ['M', 'T', 'Q', 'A', 'P'] 
Longest names: ['SPIDER-MAN/PETER PARKERKER']


## 4

In [ ]:
def not_spidey_friend(hero):
  spidey_tot = set(net_df[net_df['hero1'].isin(almost_spidy)]['hero2'].values) | set(net_df[net_df['hero2'].isin(almost_spidy)]['hero1'].values)

  hero_tot = set(net_df[net_df['hero2'] == hero]['hero1'].values) | set(net_df[net_df['hero1'] == hero]['hero2'].values)
  

  if hero in spidey_tot:
    spidey_tot.remove(hero)
    
  for i in almost_spidy:
    if i in hero_tot:
      hero_tot.remove(i)
  
  # print(len(spidey_tot))
  return len(list(filter(lambda n: n not in spidey_tot, hero_tot)))

not_spidey_friend('IRON MAN/TONY STARK')


735

## 5

In [ ]:
srch = 'CAPTAIN AMERICA'

temp = net_df[net_df['hero1'] == srch]['hero2'].values
lst = list(set(np.append(temp, net_df[net_df['hero2'] == srch]['hero1'].values)))

friends = []

for frnd in lst:
  temp = net_df[net_df['hero1'] == frnd]['hero2'].values
  fr_lst = list(set(np.append(temp, net_df[net_df['hero2'] == frnd]['hero1'].values)))
  friends = list(set(np.append(friends, fr_lst)))

print(len(friends))

6364


# **Q2**

In [ ]:
!kaggle datasets download usgs/earthquake-database  -p ./datasets/earthquake/
!unzip ./datasets/earthquake/*.zip  -d ./datasets/earthquake
earthquakes_df = pd.read_csv('/content/datasets/earthquake/database.csv')


earthquake-database.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ./datasets/earthquake/earthquake-database.zip
replace ./datasets/earthquake/database.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## 1

In [ ]:
earthquakes_df[earthquakes_df['Magnitude'] == earthquakes_df['Magnitude'].max()]

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,Magnitude Error,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
17083,12/26/2004,00:58:53,3.295,95.982,Earthquake,30.0,NaN,601.0,9.1,MW,NaN,NaN,22.0,NaN,NaN,1.17,OFFICIAL20041226005853450_30,OFFICIAL,US,OFFICIAL,Reviewed
20501,03/11/2011,05:46:24,38.297,142.373,Earthquake,29.0,NaN,541.0,9.1,MWW,NaN,NaN,9.5,NaN,NaN,1.16,OFFICIAL20110311054624120_30,OFFICIAL,US,OFFICIAL,Reviewed


## 2

In [ ]:
nuclear_explosion = earthquakes_df[earthquakes_df['Type'] == 'Nuclear Explosion']
nuclear_explosion['Magnitude'].mean()

5.850685714285718

## 3

In [ ]:
from pandas.core.common import SettingWithCopyWarning

warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

earthquakes_df['Date'] = earthquakes_df['Date'].str[-4:]
earthquakes_up_6 = earthquakes_df[earthquakes_df['Magnitude'] > 6]
earthquakes_up_6 = pd.DataFrame(earthquakes_up_6.groupby(earthquakes_up_6['Date'])['Date'].count())
earthquakes_up_9 = earthquakes_df[earthquakes_df['Magnitude'] > 9]
earthquakes_up_9 = pd.DataFrame(earthquakes_up_9.groupby(earthquakes_up_9['Date'])['Date'].count())


mx6 = earthquakes_up_6[earthquakes_up_6['Date'] == earthquakes_up_6['Date'].max()]
mx6['above'] = 6
mx9 = earthquakes_up_9[earthquakes_up_9['Date'] == earthquakes_up_9['Date'].max()]
mx9['above'] = 9

total = pd.DataFrame(pd.concat([mx6,mx9]))
total

,Date,above
Date,,
2011,163,6
2004,1,9
2011,1,9


## 4

In [ ]:
earthquakes_df['Time'] = earthquakes_df['Time'].str[:2]
hour_count = pd.DataFrame(earthquakes_df.groupby(['Time'])['Time'].count())
hour_count[hour_count['Time'] == hour_count['Time'].max()]

,Time
Time,
13,1058


# **Q3**

In [ ]:
!kaggle datasets download -d karangadiya/fifa19 -p ./datasets/fifa19/
!unzip ./datasets/fifa19/*.zip -d ./datasets/fifa19/
fifa_df = pd.read_csv('/content/datasets/fifa19/data.csv')

  0% 0.00/2.18M [00:00<?, ?B/s]
100% 2.18M/2.18M [00:00<00:00, 61.2MB/s]
Archive:  ./datasets/fifa19/fifa19.zip
  inflating: ./datasets/fifa19/data.csv  


## 1

In [ ]:
def highest_strength(country):
  pl_df = fifa_df[fifa_df['Nationality'] == country.capitalize()]
  return  pl_df[pl_df['Strength'] == pl_df['Strength'].max()]

In [ ]:
highest_strength('france')

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,...,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
1193,1193,202750,W. Boly,27,https://cdn.sofifa.org/players/4/19/202750.png,France,https://cdn.sofifa.org/flags/18.png,77,80,Wolverhampton Wanderers,https://cdn.sofifa.org/teams/2/light/110.png,€9M,€58K,1660,Right,1.0,4.0,2.0,Medium/ Medium,Normal,Yes,LCB,15.0,"Jun 9, 2018",NaN,2021,6'5,214lbs,54+2,54+2,54+2,52+2,52+2,52+2,52+2,52+2,54+2,54+2,54+2,55+2,...,68+2,76+2,76+2,76+2,68+2,50.0,32.0,75.0,70.0,23.0,48.0,43.0,35.0,58.0,60.0,57.0,69.0,55.0,71.0,45.0,52.0,62.0,58.0,93.0,21.0,79.0,76.0,43.0,52.0,56.0,76.0,72.0,78.0,78.0,7.0,8.0,15.0,10.0,9.0,€17.8M
5192,5192,228732,H. Moukoudi,20,https://cdn.sofifa.org/players/4/19/228732.png,France,https://cdn.sofifa.org/flags/18.png,70,81,Le Havre AC,https://cdn.sofifa.org/teams/2/light/1738.png,€2.7M,€3K,1435,Right,1.0,3.0,2.0,Medium/ Medium,Stocky,No,RCB,4.0,"Jul 1, 2016",NaN,2019,6'3,196lbs,47+2,47+2,47+2,41+2,43+2,43+2,43+2,41+2,43+2,43+2,43+2,44+2,...,61+2,69+2,69+2,69+2,61+2,23.0,26.0,74.0,55.0,27.0,39.0,30.0,29.0,46.0,55.0,51.0,62.0,40.0,60.0,38.0,42.0,52.0,73.0,93.0,25.0,61.0,67.0,31.0,28.0,38.0,60.0,69.0,72.0,67.0,12.0,14.0,12.0,15.0,9.0,€5.3M
7144,7144,200067,Z. Diallo,31,https://cdn.sofifa.org/players/4/19/200067.png,France,https://cdn.sofifa.org/flags/18.png,68,68,Montreal Impact,https://cdn.sofifa.org/teams/2/light/111139.png,€600K,€3K,1502,Right,1.0,3.0,2.0,Medium/ Medium,Normal,No,CB,5.0,"Feb 7, 2018",NaN,2019,6'4,198lbs,49+2,49+2,49+2,46+2,46+2,46+2,46+2,46+2,46+2,46+2,46+2,48+2,...,59+2,67+2,67+2,67+2,59+2,48.0,35.0,63.0,56.0,35.0,46.0,52.0,45.0,47.0,52.0,46.0,39.0,39.0,59.0,30.0,53.0,39.0,64.0,93.0,38.0,75.0,64.0,42.0,41.0,48.0,64.0,67.0,70.0,69.0,7.0,8.0,10.0,9.0,13.0,€900K
8504,8504,212420,M. Sarr,24,https://cdn.sofifa.org/players/4/19/212420.png,France,https://cdn.sofifa.org/flags/18.png,67,73,Charlton Athletic,https://cdn.sofifa.org/teams/2/light/89.png,€900K,€4K,1428,Left,1.0,3.0,2.0,Medium/ Medium,Normal,No,LB,23.0,"Jul 28, 2015",NaN,2020,6'5,196lbs,46+2,46+2,46+2,44+2,45+2,45+2,45+2,44+2,47+2,47+2,47+2,47+2,...,57+2,66+2,66+2,66+2,57+2,36.0,26.0,70.0,67.0,31.0,49.0,29.0,34.0,55.0,51.0,34.0,51.0,34.0,57.0,33.0,45.0,53.0,60.0,93.0,28.0,64.0,58.0,31.0,51.0,42.0,58.0,66.0,66.0,64.0,14.0,12.0,8.0,8.0,8.0,€1.8M


In [ ]:
highest_strength('brazil')

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,...,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
1124,1124,232381,Wesley,21,https://cdn.sofifa.org/players/4/19/232381.png,Brazil,https://cdn.sofifa.org/flags/54.png,77,85,Club Brugge KV,https://cdn.sofifa.org/teams/2/light/231.png,€13.5M,€20K,1831,Right,1.0,3.0,3.0,High/ High,Stocky,No,LS,7.0,"Jan 29, 2016",NaN,2023,6'3,207lbs,75+2,75+2,75+2,72+2,74+2,74+2,74+2,72+2,72+2,72+2,72+2,72+2,...,52+2,52+2,52+2,52+2,52+2,65.0,75.0,73.0,75.0,71.0,76.0,57.0,49.0,67.0,78.0,68.0,79.0,50.0,71.0,45.0,81.0,66.0,74.0,95.0,67.0,80.0,37.0,74.0,68.0,60.0,75.0,35.0,27.0,15.0,12.0,13.0,14.0,8.0,6.0,€23M


## 2

In [ ]:
def overall_avg(club):
  cl_df = fifa_df[fifa_df['Club'] == club]
  return  cl_df['Overall'].mean(axis=0, skipna=True)

In [ ]:
overall_avg('FC Barcelona')

78.03030303030303